In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

In [2]:
def psnr(y_true, y_pred):
    """
    Custom PSNR (Peak Signal to Noise Ratio) metric.

    Parameters:
    y_true (tf.Tensor): The ground truth image.
    y_pred (tf.Tensor): The predicted image.

    Returns:
    tf.Tensor: PSNR value.
    """
    max_pixel = 1.0
    mse = K.mean(K.square(y_pred - y_true))
    psnr = 20 * K.log(max_pixel / K.sqrt(mse)) / K.log(10.0)
    return psnr

In [1]:
def build_vdsrcnn_model_LR2_HR(numLayers = 18, input_shape=(None, None, 3)):
    """
    Builds the SRCNN model.

    Parameters:
    input_shape (tuple): Shape of the input image. Default is (None, None, 3) for images with arbitrary dimensions.

    Returns:
    model (tf.keras.Model): SRCNN model.
    """
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
    for _ in range(numLayers):
        model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(filters=3, kernel_size=(3, 3), activation='linear', padding='same'))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[psnr])

    return model

In [2]:
def trainModel(model, trainIn, trainOut, valIn, valOut, epochs, batch, device):
    early_stopping = EarlyStopping(monitor='val_psnr', patience=3, mode='max', verbose=1)
    if device=="GPU":
        with tf.device("/GPU:0"):
            history = model.fit(trainIn, trainOut, epochs=epochs, batch_size=batch, validation_data=(valIn, valOut), callbacks=[early_stopping])
    else:
        with tf.device("/CPU:0"):
            history = model.fit(trainIn, trainOut, epochs=epochs, batch_size=batch, validation_data=(valIn, valOut), callbacks=[early_stopping])
    return history